In [40]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier
import gc
import time
import random

In [41]:
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (12, 10) 
plt.style.use('ggplot')

In [42]:
transactions = pd.read_csv('../../data_mini/transactions.csv')

In [43]:
q = transactions['mcc_code'].value_counts()
valid_mcc = list(q[q<5000].index)
transactions.loc[transactions['mcc_code'].isin(valid_mcc), 'mcc_code'] = 9999

In [44]:
transactions.head()

,user_id,mcc_code,transaction_amt,transaction_dttm
0,20406,5411,-361,2020-08-03 08:05:23
1,20406,5499,-137,2020-08-05 01:27:40
2,20406,5499,-138,2020-08-05 03:28:11
3,20406,4829,-309,2020-08-06 00:36:29
4,20406,5411,-133,2020-08-09 00:30:13


In [45]:
clickstream = pd.read_csv('../../data_mini/clickstream.csv')

In [46]:
q = clickstream['cat_id'].value_counts()
valid_cats = q[q<5000].index
clickstream.loc[clickstream['cat_id'].isin(valid_cats), 'cat_id'] = 9998

In [47]:
clickstream.head()

,user_id,cat_id,timestamp
0,6877,165,2021-01-30 20:08:12
1,6877,165,2021-01-31 20:06:29
2,6877,308,2021-01-31 20:12:00
3,6877,931,2021-01-31 22:12:00
4,6877,931,2021-02-01 16:57:00


In [48]:
print(transactions['mcc_code'].nunique(), clickstream['cat_id'].nunique())

123 112


In [49]:
Z = pd.read_csv('../../data_mini/train_matching_puzzle_folded.csv')
Z.head()

,bank,rtk,target,fold
0,6090,16219,1,0
1,21920,16282,1,2
2,15668,18578,1,1
3,17254,13760,1,0
4,14856,9899,1,2


In [50]:
transactions = transactions[transactions['user_id'].isin(list(Z['bank']))]
clickstream = clickstream[clickstream['user_id'].isin(list(Z['rtk']))]

In [51]:
transactions['transaction_amt'] = abs(transactions['transaction_amt'])
transactions['transaction_dttm'] = pd.to_datetime(transactions['transaction_dttm'])
transactions['hour'] = transactions['transaction_dttm'].dt.hour

#кол-во минут с начала суток
transactions['sec1'] = transactions['hour'] * 60 + transactions['transaction_dttm'].dt.minute

In [52]:
bankclient_embed = transactions.pivot_table(index = ['user_id'],
                            values=['sec1'],
                            columns=['mcc_code'],
                            aggfunc=['count', 'median']).fillna(0)
bankclient_embed.columns = ['v1_' + str(col[0]) + '-' + str(col[2]) for col in bankclient_embed.columns]

In [53]:
be = transactions.pivot_table(index = ['user_id'],
                            values=['transaction_amt'],
                            columns=['mcc_code'],
                            aggfunc=['sum']).fillna(0)
be.columns = ['v1_' + str(col[0]) + '-' + str(col[2]) for col in be.columns]

bankclient_embed = bankclient_embed.merge(be, how='left', left_index=True, right_index=True)

In [54]:
bankclient_embed['mcc_total'] = transactions.groupby('user_id')['mcc_code'].sum()

In [55]:
for col in bankclient_embed.columns:
    if 'count' in col:
        bankclient_embed[col+'_sh'] = bankclient_embed[col] / bankclient_embed['mcc_total']

In [56]:
transactions_by_hour = pd.pivot_table(transactions, index=['user_id'], columns='hour', values='transaction_amt', aggfunc='count').fillna(0).astype('int32')
transactions_by_hour['tr_sum'] = transactions_by_hour.sum(axis=1)
for col in transactions_by_hour.columns[:-1]:
    transactions_by_hour[col] /= transactions_by_hour['tr_sum']

del transactions_by_hour['tr_sum']
transactions_by_hour.columns = ['trans_h_'+ str(col) + '_sh' for col in transactions_by_hour.columns]

In [57]:
del transactions
gc.collect()

40

In [58]:
clickstream['timestamp'] = pd.to_datetime(clickstream['timestamp'])
clickstream['hour'] = clickstream['timestamp'].dt.hour

#кол-во минут с начала суток
clickstream['sec2'] = clickstream['hour']*60 + clickstream['timestamp'].dt.minute

In [59]:
clickstream_embed = clickstream.pivot_table(index = ['user_id'],
                        values=['sec2'],
                        columns=['cat_id'],
                        aggfunc=['count', 'median']).fillna(0)

clickstream_embed.columns = ['v2_' + str(col[0]) + '-' + str(col[2]) for col in clickstream_embed.columns]

In [60]:
clickstream_embed['cat_total'] = clickstream.groupby('user_id')['cat_id'].sum()

In [61]:
for col in clickstream_embed.columns:
    if 'count' in col:
        clickstream_embed[col+'_sh'] = clickstream_embed[col] / clickstream_embed['cat_total']

In [62]:
clicks_by_hour = pd.pivot_table(clickstream, index='user_id', columns='hour', values='timestamp', aggfunc='count').fillna(0).astype('int32')
clicks_by_hour['cl_sum'] = clicks_by_hour.sum(axis=1)
for col in clicks_by_hour.columns[:-1]:
    clicks_by_hour[col] /= clicks_by_hour['cl_sum']

del clicks_by_hour['cl_sum']
clicks_by_hour.columns = ['click_h_'+ str(col) + '_sh' for col in clicks_by_hour.columns]

In [63]:
del clickstream
gc.collect()

40

In [64]:
def df_compress(df):
    dtype_df = list()
    for col, tp in zip(df.columns.tolist(), df.dtypes.tolist()):
        if '_sh' in col:
            dtype_df.append('float32')
        elif 'count' in col:
            dtype_df.append('int32')    
        elif 'mean' in col:
            dtype_df.append('int32')
        elif 'median' in col:
            dtype_df.append('int32')
        elif 'sum' in col:
            dtype_df.append('int64')
        elif col in ['bank', 'rtk', 'target', 'fold']:
            dtype_df.append('int16')
        elif tp in ['int64', 'int32']:
            dtype_df.append('int32')
        elif tp in ['float64', 'float32']:
            dtype_df.append('float32')
        else:
            dtype_df.append('object')
    
    dtype_df = dict(zip(df.columns.tolist(),dtype_df))
    df = df.astype(dtype_df)
    return df

In [65]:
bankclient_embed = bankclient_embed.merge(transactions_by_hour, how='left', left_index=True, right_index=True)
clickstream_embed = clickstream_embed.merge(clicks_by_hour, how='left', left_index=True, right_index=True)

In [66]:
clickstream_embed = df_compress(clickstream_embed)

In [67]:
bankclient_embed = df_compress(bankclient_embed)

In [68]:
bankclient_embed = bankclient_embed.reset_index()
clickstream_embed = clickstream_embed.reset_index()
bankclient_embed.rename(columns={'user_id': 'bank'}, inplace=True)
clickstream_embed.rename(columns={'user_id': 'rtk'}, inplace=True)

In [69]:
clickstream_embed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14671 entries, 0 to 14670
Columns: 362 entries, rtk to click_h_23_sh
dtypes: float32(136), int32(225), int64(1)
memory usage: 20.3 MB


### CV

In [70]:
# min-max scaler by rtk
def nxscaler(df, col):
    g = df.groupby('rtk')[col]
    min_, max_ = g.transform('min'), g.transform('max')
    df[col+'_scaled'] = (df[col] - min_) / (max_ - min_)
    return df

In [86]:
def score(df):
    df = df.sort_values(['pred'], ascending=False)
    df100 = df.groupby('bank').head(100).copy()
    df100['rank'] = df100.groupby('bank').cumcount()
    df100['mrr'] = df100['target'] * (1 / (1 + df100['rank']))
    q = df100.groupby('bank')['mrr'].max()
    precision = len(q[q > 0]) / len(q)
    mrr = q.mean()
    r1 = 2 * precision * mrr / (precision + mrr)
    return {'R1': round(r1, 4), 'MRR': round(mrr, 4), 'precision': round(precision, 4)}

In [72]:
%%time

ALL_FOLDS = pd.DataFrame()
for fold in range(3):
    train, test = Z[Z['fold'] != fold], Z[Z['fold'] == fold]
    print(train.shape[0], test.shape[0])
    
    b_embed_train = bankclient_embed[bankclient_embed['bank'].isin(train['bank'])]
    b_embed_test = bankclient_embed[bankclient_embed['bank'].isin(test['bank'])]

    c_embed_train = clickstream_embed[clickstream_embed['rtk'].isin(train['rtk'])]
    c_embed_test = clickstream_embed[clickstream_embed['rtk'].isin(test['rtk'])]
    
    #### train set
    N = 9
    
    bank_ids = list(train['bank'])
    rtk_ids = list(train['rtk'])
    part_of_submit = pd.DataFrame(bank_ids, columns=['bank'])
    part_of_submit['rtk'] = part_of_submit['bank'].apply(lambda x: random.sample(rtk_ids, N))
    part_of_submit = part_of_submit.explode('rtk').astype('int16')
    
    part_of_submit['target'] = 0
    part_of_submit = part_of_submit.append(train.drop('fold', axis=1), ignore_index=True)
    part_of_submit = part_of_submit.drop_duplicates(['bank', 'rtk'], keep='last')
    
    part_of_submit = part_of_submit.merge(b_embed_train, how='left', on=['bank']).merge(c_embed_train, how='left', on=['rtk'])
    
    X_train = part_of_submit.drop(['bank', 'rtk', 'target'], axis=1)
    y_train = part_of_submit['target']
    
    #### train catboost
    clf = CatBoostClassifier(
        iterations=3500,
        learning_rate=0.03,
        verbose=350,
    )
    
    clf.fit(X_train, y_train)
    
    #### ptredict on test
    batch_num = 500
    K = len(b_embed_test['bank'])//batch_num
 
    FOLD = pd.DataFrame()
    bank_ids = list(test['bank'])
    rtk_ids = list(test['rtk'])
    for i in range(K+1):
        this_bank_ids = bank_ids[i*batch_num:(i+1)*batch_num]
        ps = pd.DataFrame(this_bank_ids, columns=['bank'])
        ps['rtk'] = ps['bank'].apply(lambda x: rtk_ids)
        ps = ps.explode('rtk').astype('int16')
        ps = ps.merge(b_embed_test, how='left', on=['bank']).merge(c_embed_test, how='left', on=['rtk'])

        tdf = ps[['bank', 'rtk']].copy()
        
        tdf['pred'] = clf.predict_proba(ps.drop(['bank', 'rtk'], axis=1))[:,1]
        FOLD = FOLD.append(tdf, ignore_index=True)

    FOLD['fold'] = fold
    ALL_FOLDS = ALL_FOLDS.append(FOLD, ignore_index=True)

9780 4891
0:	learn: 0.6672702	total: 427ms	remaining: 24m 55s
350:	learn: 0.2849831	total: 37.9s	remaining: 5m 39s
700:	learn: 0.2664235	total: 1m 13s	remaining: 4m 54s
1050:	learn: 0.2532016	total: 1m 49s	remaining: 4m 13s
1400:	learn: 0.2420058	total: 2m 24s	remaining: 3m 36s
1750:	learn: 0.2324575	total: 2m 59s	remaining: 2m 59s
2100:	learn: 0.2237240	total: 3m 34s	remaining: 2m 22s
2450:	learn: 0.2157923	total: 4m 9s	remaining: 1m 46s
2800:	learn: 0.2081138	total: 4m 46s	remaining: 1m 11s
3150:	learn: 0.2012791	total: 5m 26s	remaining: 36.1s
3499:	learn: 0.1944513	total: 6m 9s	remaining: 0us
9781 4890
0:	learn: 0.6672671	total: 159ms	remaining: 9m 15s
350:	learn: 0.2844638	total: 44.1s	remaining: 6m 35s
700:	learn: 0.2651734	total: 1m 28s	remaining: 5m 53s
1050:	learn: 0.2515776	total: 2m 7s	remaining: 4m 57s
1400:	learn: 0.2409146	total: 2m 44s	remaining: 4m 7s
1750:	learn: 0.2312260	total: 3m 22s	remaining: 3m 22s
2100:	learn: 0.2222714	total: 3m 58s	remaining: 2m 38s
2450:	learn

In [78]:
ALL_FOLDS = pd.merge(ALL_FOLDS, Z[['bank', 'rtk', 'target']], how='left', on=['bank', 'rtk'])
ALL_FOLDS['target'] = ALL_FOLDS['target'].fillna(0).astype(int)

ALL_FOLDS = nxscaler(ALL_FOLDS, 'pred')
ALL_FOLDS['pred_h'] = ALL_FOLDS['pred_scaled'] * ALL_FOLDS['pred'] / (ALL_FOLDS['pred_scaled'] + ALL_FOLDS['pred'])

In [87]:
# top 100 by pred
score(ALL_FOLDS)

{'R1': 0.0431, 'MRR': 0.024, 'precision': 0.2089}

In [85]:
# top 100 by pred_h (min-max scaled)
score(ALL_FOLDS)

{'R1': 0.046, 'MRR': 0.0258, 'precision': 0.2146}